# Wav2Vec2 Model 

## Environment Setup 

* Clean and isolated Python environment in order to manage packages and avoid conflicts


In [26]:
# creating virtual enviornment
!python3 -m venv wavenv

In [27]:
# activating wavnev enviornment - anything ran inside will stay inside
!source wavenv/bin/activate && echo "Virtual environment activated"

Virtual environment activated


In [28]:
# installs so jupyter can recognize kernel option 
!wavenv/bin/pip install ipykernel
!wavenv/bin/python -m ipykernel install --user --name=wavenv --display-name "Python (wavenv)

/bin/bash: -c: line 0: unexpected EOF while looking for matching `"'
/bin/bash: -c: line 1: syntax error: unexpected end of file


In [29]:
#checking python running from my virtual environment 
import sys
print(sys.executable)

/stor/home/spl742/wavenv/bin/python


In [30]:
# installing packages built for ROCm 5.7
!pip install --upgrade --force-reinstall --no-cache-dir torch torchvision torchaudio --index-url https://download.pytorch.org/whl/rocm5.7

Looking in indexes: https://download.pytorch.org/whl/rocm5.7
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 GB 100.1 MB/s eta 0:00:0000:010:02m
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 MB 43.8 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 21.5 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.2/6.2 MB 226.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 234.3/234.3 MB 209.9 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 214.8 MB/s eta 0:00:00a 0:00:01
INFO: pip is looking at multiple versions of networkx to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 157.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.2/536.2 kB 217.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 213.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━

In [31]:
# GPU check
import torch

print("CUDA available:", torch.cuda.is_available())
print("Device count:", torch.cuda.device_count())
print("Device name:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "CPU")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

CUDA available: True
Device count: 8
Device name: AMD Instinct MI100


In [32]:
# library imports 

import os
import numpy as np
import zipfile
import librosa
import soundfile as sf
import torch
import torchaudio
from transformers import Wav2Vec2Processor, Wav2Vec2Model
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import train_test_split
from torch import nn
from torch.utils.data import DataLoader, TensorDataset
import glob

# device check
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)


Using device: cuda


## Wav2Vec2

Wav2Vec is a pre-trained model developed by Facebook AI. Especially designed to extract meaningful features from raw audio without manual feature eng. Including:

* Self Supervised Learning on Speech: trained on real human speech data. Able to pickup subtle patterns in how real people speak. 

* Feature Extraction: Is able to output high deimensional embeddings -- strong input features for a classifier

* Transfer Learning: No fine tuning

* Detection: Wav2Vec can help identify patterns or rythm, Wav2Vec2 helps uncover details 

* Block loads pre-trained Wav2Vec model and sets everything to run in CPU:

* processor = loads feature extractor/tokenizer that matches the pre-trained model. Processor handles converting raw audio into a format for Wav2Vec 

* model = loading model weights -- trained on 960 hrs of unlabeled speech data 

In [33]:
#Wav2Vec2 model 
processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-large-960h")
model = Wav2Vec2Model.from_pretrained("facebook/wav2vec2-large-960h").cpu()  # Force CPU
device = torch.device("cpu")


Some weights of Wav2Vec2Model were not initialized from the model checkpoint at facebook/wav2vec2-large-960h and are newly initialized: ['masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## Data Preprocessing

In [35]:
# unzipping audio data 

# ZIP files into folders
zip_path = 'FAKE.zip'
real_zip_path = 'REAL.zip'
extract_path = 'fake_audio/'

# directory check
os.makedirs(extract_path, exist_ok=True)

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

with zipfile.ZipFile(real_zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)


In [36]:

# merging audio segments for each speaker

base_path = 'fake_audio/REAL'  

# loop through each folder
# collecting all .wav files inside 

for folder_name in os.listdir(base_path):
    folder_path = os.path.join(base_path, folder_name)

    if os.path.isdir(folder_path):
        print(f"Merging .wav files in: {folder_name}")
        combined_audio = []
        sr = None

        wav_files = sorted([f for f in os.listdir(folder_path) if f.endswith('.wav')])

        # librosa - loading each file and appending the audio data to list 
        for wav_file in wav_files:
            wav_path = os.path.join(folder_path, wav_file)
            y, sr = librosa.load(wav_path, sr=None)
            combined_audio.append(y)

        if combined_audio:
            merged_audio = np.concatenate(combined_audio) # merging clips into one wav
            output_path = os.path.join(base_path, f"{folder_name}_merged.wav")
            sf.write(output_path, merged_audio, sr) # save into new .wav - using soundfile
            print(f"Saved: {output_path}") 
        else:
            print(f"No .wav files in {folder_name}")


Merging .wav files in: obama-original
Saved: fake_audio/REAL/obama-original_merged.wav
Merging .wav files in: trump-original
Saved: fake_audio/REAL/trump-original_merged.wav
Merging .wav files in: biden-original
Saved: fake_audio/REAL/biden-original_merged.wav
Merging .wav files in: ryan-original
Saved: fake_audio/REAL/ryan-original_merged.wav
Merging .wav files in: musk-original
Saved: fake_audio/REAL/musk-original_merged.wav
Merging .wav files in: linus-original
Saved: fake_audio/REAL/linus-original_merged.wav
Merging .wav files in: taylor-original
Saved: fake_audio/REAL/taylor-original_merged.wav
Merging .wav files in: margot-original
Saved: fake_audio/REAL/margot-original_merged.wav


In [12]:
# merging process repeated, to combine fake audio segments into single .wav files 

base_path = 'fake_audio/FAKE'  # for FAKE folder

for folder_name in os.listdir(base_path):
    folder_path = os.path.join(base_path, folder_name)

    if os.path.isdir(folder_path):
        print(f"Merging .wav files in: {folder_name}")
        combined_audio = []
        sr = None

        wav_files = sorted([f for f in os.listdir(folder_path) if f.endswith('.wav')])

        for wav_file in wav_files:
            wav_path = os.path.join(folder_path, wav_file)
            y, sr = librosa.load(wav_path, sr=None)
            combined_audio.append(y)

        if combined_audio:
            merged_audio = np.concatenate(combined_audio)
            output_path = os.path.join(base_path, f"{folder_name}_merged.wav")
            sf.write(output_path, merged_audio, sr)
            print(f"Saved: {output_path}")
        else:
            print(f"No .wav files in {folder_name}")


Merging .wav files in: musk-to-taylor
Saved: fake_audio/FAKE/musk-to-taylor_merged.wav
Merging .wav files in: obama-to-taylor
Saved: fake_audio/FAKE/obama-to-taylor_merged.wav
Merging .wav files in: Trump
No .wav files in Trump
Merging .wav files in: musk-to-obama
Saved: fake_audio/FAKE/musk-to-obama_merged.wav
Merging .wav files in: taylor-to-biden
Saved: fake_audio/FAKE/taylor-to-biden_merged.wav
Merging .wav files in: biden-to-ryan
Saved: fake_audio/FAKE/biden-to-ryan_merged.wav
Merging .wav files in: biden-to-Obama
Saved: fake_audio/FAKE/biden-to-Obama_merged.wav
Merging .wav files in: linus-to-margot
Saved: fake_audio/FAKE/linus-to-margot_merged.wav
Merging .wav files in: trump-to-musk
Saved: fake_audio/FAKE/trump-to-musk_merged.wav
Merging .wav files in: musk-to-linus
Saved: fake_audio/FAKE/musk-to-linus_merged.wav
Merging .wav files in: ryan-to-trump
Saved: fake_audio/FAKE/ryan-to-trump_merged.wav
Merging .wav files in: taylor-to-obama
Saved: fake_audio/FAKE/taylor-to-obama_merg

## Wav2Vec2 feature extraction function

In [37]:
# processes a single .wav audio file and extracts features using the Wav2Vec2 model

def extract_features(file_path, max_duration_sec=10):
    try:
        print(f"Processing: {file_path}")
        
        # loading audio waveform + sample rate
        audio_input, sample_rate = torchaudio.load(file_path)
        print(f"Loaded audio: {audio_input.shape}, Sample rate: {sample_rate}")

        # truncate long files - consistent inputs
        max_len = int(sample_rate * max_duration_sec)
        if audio_input.shape[1] > max_len:
            audio_input = audio_input[:, :max_len]

        # convert stereo to mono if audio has more than one channel
        if audio_input.shape[0] > 1:
            audio_input = audio_input.mean(dim=0, keepdim=True)
            print("Converted to mono")

        # resample to 16kHz - wav2vec2 expected input
        if sample_rate != 16000:
            print(f"Resampling from {sample_rate} to 16000 Hz")
            resampler = torchaudio.transforms.Resample(orig_freq=sample_rate, new_freq=16000)
            audio_input = resampler(audio_input)

        # preparing model input for wav2vec2 
        input_values = processor(audio_input.squeeze().numpy(), sampling_rate=16000, return_tensors="pt", padding=True)
        input_values = {k: v.to("cpu") for k, v in input_values.items()}
        print("Prepared input for model")

        # extracting features 
        # output: hidden states - averaged over time, produces a single feature vector 
        with torch.no_grad():
            outputs = model(**input_values)
            features = outputs.last_hidden_state
            print("Features extracted successfully")

        # feature vector - numpy array (summary of audio characteristics)
        return features.mean(dim=1).squeeze().cpu().numpy()

    except Exception as e:
        print(f"Error processing {file_path}:", e)
        return None

    # included print statements for debugging! 

In [38]:
# test block 

# specific block 
test_path = "fake_audio/FAKE/biden-to-Trump/segment_2414.wav"
feature = extract_features(test_path) # feature extraction

# output
if feature is not None:
    print("Feature shape:", feature.shape)
else:
    print("Feature extraction failed.")


Processing: fake_audio/FAKE/biden-to-Trump/segment_2414.wav
Loaded audio: torch.Size([2, 200000]), Sample rate: 40000
Converted to mono
Resampling from 40000 to 16000 Hz
Prepared input for model
Features extracted successfully
Feature shape: (1024,)


## Batch feature extraction

In [39]:
# libraries 
from pathlib import Path
import numpy as np
import os

# input/output paths - where to save features
real_base = Path("fake_audio/REAL")
fake_base = Path("fake_audio/FAKE")
real_out = Path("features/real")
fake_out = Path("features/fake")

# creating output directories if they don't exist
real_out.mkdir(parents=True, exist_ok=True)
fake_out.mkdir(parents=True, exist_ok=True)

# define a processing function with skip logic (easier to pause and resume without having to start over)
def process_and_save(base_path, out_path):
    for folder in base_path.iterdir():
        if folder.is_dir():
            for wav_file in folder.glob("*.wav"):
                save_name = f"{folder.name}_{wav_file.name.replace('.wav', '.npy')}"
                save_path = out_path / save_name

                # skip if file already processed
                if save_path.exists():
                    print(f"Skipping already processed: {save_path}")
                    continue

                # extract features left 
                features = extract_features(str(wav_file))
                if features is not None:
                    np.save(save_path, features)
                    print(f"Saved features to: {save_path}")

# call function on both real and fake data folders
process_and_save(real_base, real_out)
process_and_save(fake_base, fake_out)


Skipping already processed: features/real/obama-original_segment_5660.npy
Skipping already processed: features/real/obama-original_segment_5667.npy
Skipping already processed: features/real/obama-original_segment_5615.npy
Skipping already processed: features/real/obama-original_segment_5669.npy
Skipping already processed: features/real/obama-original_segment_5731.npy
Skipping already processed: features/real/obama-original_segment_5703.npy
Skipping already processed: features/real/obama-original_segment_5627.npy
Skipping already processed: features/real/obama-original_segment_5620.npy
Skipping already processed: features/real/obama-original_segment_5704.npy
Skipping already processed: features/real/obama-original_segment_5683.npy
Skipping already processed: features/real/obama-original_segment_5652.npy
Skipping already processed: features/real/obama-original_segment_5684.npy
Skipping already processed: features/real/obama-original_segment_5655.npy
Skipping already processed: features/r

Error processing fake_audio/REAL/biden-original/segment_5370.wav: Calculated padded input size per channel: (1). Kernel size: (3). Kernel size can't be greater than actual input size
Skipping already processed: features/real/biden-original_segment_5282.npy
Skipping already processed: features/real/biden-original_segment_5253.npy
Skipping already processed: features/real/biden-original_segment_5342.npy
Skipping already processed: features/real/biden-original_segment_5266.npy
Skipping already processed: features/real/biden-original_segment_5261.npy
Skipping already processed: features/real/biden-original_segment_5339.npy
Skipping already processed: features/real/biden-original_segment_5345.npy
Skipping already processed: features/real/biden-original_segment_5337.npy
Skipping already processed: features/real/biden-original_segment_5268.npy
Skipping already processed: features/real/biden-original_segment_5330.npy
Skipping already processed: features/real/biden-original_segment_5321.npy
Ski

Skipping already processed: features/fake/biden-to-Obama_segment_2927.npy
Skipping already processed: features/fake/biden-to-Obama_segment_3002.npy
Skipping already processed: features/fake/biden-to-Obama_segment_3037.npy
Skipping already processed: features/fake/biden-to-Obama_segment_3030.npy
Skipping already processed: features/fake/biden-to-Obama_segment_2969.npy
Skipping already processed: features/fake/biden-to-Obama_segment_3042.npy
Skipping already processed: features/fake/biden-to-Obama_segment_2967.npy
Skipping already processed: features/fake/biden-to-Obama_segment_2960.npy
Skipping already processed: features/fake/biden-to-Obama_segment_3039.npy
Skipping already processed: features/fake/biden-to-Obama_segment_2994.npy
Skipping already processed: features/fake/biden-to-Obama_segment_2945.npy
Skipping already processed: features/fake/biden-to-Obama_segment_2939.npy
Skipping already processed: features/fake/biden-to-Obama_segment_2993.npy
Skipping already processed: features/f

Prepared input for model
Features extracted successfully
Saved features to: features/fake/linus-to-margot_segment_2610.npy
Processing: fake_audio/FAKE/linus-to-margot/segment_2575.wav
Loaded audio: torch.Size([2, 220500]), Sample rate: 44100
Converted to mono
Resampling from 44100 to 16000 Hz
Prepared input for model
Features extracted successfully
Saved features to: features/fake/linus-to-margot_segment_2575.npy
Processing: fake_audio/FAKE/linus-to-margot/segment_2662.wav
Loaded audio: torch.Size([2, 220500]), Sample rate: 44100
Converted to mono
Resampling from 44100 to 16000 Hz
Prepared input for model
Features extracted successfully
Saved features to: features/fake/linus-to-margot_segment_2662.npy
Processing: fake_audio/FAKE/linus-to-margot/segment_2619.wav
Loaded audio: torch.Size([2, 220500]), Sample rate: 44100
Converted to mono
Resampling from 44100 to 16000 Hz
Prepared input for model
Features extracted successfully
Saved features to: features/fake/linus-to-margot_segment_2619

Prepared input for model
Features extracted successfully
Saved features to: features/fake/linus-to-margot_segment_2658.npy
Processing: fake_audio/FAKE/linus-to-margot/segment_2624.wav
Loaded audio: torch.Size([2, 220500]), Sample rate: 44100
Converted to mono
Resampling from 44100 to 16000 Hz
Prepared input for model
Features extracted successfully
Saved features to: features/fake/linus-to-margot_segment_2624.npy
Processing: fake_audio/FAKE/linus-to-margot/segment_2656.wav
Loaded audio: torch.Size([2, 220500]), Sample rate: 44100
Converted to mono
Resampling from 44100 to 16000 Hz
Prepared input for model
Features extracted successfully
Saved features to: features/fake/linus-to-margot_segment_2656.npy
Processing: fake_audio/FAKE/linus-to-margot/segment_2590.wav
Loaded audio: torch.Size([2, 220500]), Sample rate: 44100
Converted to mono
Resampling from 44100 to 16000 Hz
Prepared input for model
Features extracted successfully
Saved features to: features/fake/linus-to-margot_segment_2590

...

Prepared input for model
Features extracted successfully
Saved features to: features/fake/biden-to-musk_segment_2732.npy
Processing: fake_audio/FAKE/biden-to-musk/segment_2740.wav
Loaded audio: torch.Size([2, 220500]), Sample rate: 44100
Converted to mono
Resampling from 44100 to 16000 Hz
Prepared input for model
Features extracted successfully
Saved features to: features/fake/biden-to-musk_segment_2740.npy
Processing: fake_audio/FAKE/biden-to-musk/segment_2791.wav
Loaded audio: torch.Size([2, 220500]), Sample rate: 44100
Converted to mono
Resampling from 44100 to 16000 Hz
Prepared input for model
Features extracted successfully
Saved features to: features/fake/biden-to-musk_segment_2791.npy
Processing: fake_audio/FAKE/biden-to-musk/segment_2747.wav
Loaded audio: torch.Size([2, 220500]), Sample rate: 44100
Converted to mono
Resampling from 44100 to 16000 Hz
Prepared input for model
Features extracted successfully
Saved features to: features/fake/biden-to-musk_segment_2747.npy
Processin

Prepared input for model
Features extracted successfully
Saved features to: features/fake/musk-to-margot_segment_2033.npy
Processing: fake_audio/FAKE/musk-to-margot/segment_2034.wav
Loaded audio: torch.Size([2, 220500]), Sample rate: 44100
Converted to mono
Resampling from 44100 to 16000 Hz
Prepared input for model
Features extracted successfully
Saved features to: features/fake/musk-to-margot_segment_2034.npy
Processing: fake_audio/FAKE/musk-to-margot/segment_2048.wav
Loaded audio: torch.Size([2, 220500]), Sample rate: 44100
Converted to mono
Resampling from 44100 to 16000 Hz
Prepared input for model
Features extracted successfully
Saved features to: features/fake/musk-to-margot_segment_2048.npy
Processing: fake_audio/FAKE/musk-to-margot/segment_1991.wav
Loaded audio: torch.Size([2, 220500]), Sample rate: 44100
Converted to mono
Resampling from 44100 to 16000 Hz
Prepared input for model
Features extracted successfully
Saved features to: features/fake/musk-to-margot_segment_1991.npy
Pr

Features extracted successfully
Saved features to: features/fake/musk-to-margot_segment_2047.npy
Processing: fake_audio/FAKE/musk-to-margot/segment_2035.wav
Loaded audio: torch.Size([2, 220500]), Sample rate: 44100
Converted to mono
Resampling from 44100 to 16000 Hz
Prepared input for model
Features extracted successfully
Saved features to: features/fake/musk-to-margot_segment_2035.npy
Processing: fake_audio/FAKE/musk-to-margot/segment_2049.wav
Loaded audio: torch.Size([2, 220500]), Sample rate: 44100
Converted to mono
Resampling from 44100 to 16000 Hz
Prepared input for model
Features extracted successfully
Saved features to: features/fake/musk-to-margot_segment_2049.npy
Processing: fake_audio/FAKE/musk-to-margot/segment_1999.wav
Loaded audio: torch.Size([2, 220500]), Sample rate: 44100
Converted to mono
Resampling from 44100 to 16000 Hz
Prepared input for model
Features extracted successfully
Saved features to: features/fake/musk-to-margot_segment_1999.npy
Processing: fake_audio/FAKE

Prepared input for model
Features extracted successfully
Saved features to: features/fake/musk-to-margot_segment_2067.npy
Processing: fake_audio/FAKE/musk-to-margot/segment_2069.wav
Loaded audio: torch.Size([2, 220500]), Sample rate: 44100
Converted to mono
Resampling from 44100 to 16000 Hz
Prepared input for model
Features extracted successfully
Saved features to: features/fake/musk-to-margot_segment_2069.npy
Processing: fake_audio/FAKE/musk-to-margot/segment_2015.wav
Loaded audio: torch.Size([2, 220500]), Sample rate: 44100
Converted to mono
Resampling from 44100 to 16000 Hz
Prepared input for model
Features extracted successfully
Saved features to: features/fake/musk-to-margot_segment_2015.npy
Processing: fake_audio/FAKE/musk-to-margot/segment_2012.wav
Loaded audio: torch.Size([2, 220500]), Sample rate: 44100
Converted to mono
Resampling from 44100 to 16000 Hz
Prepared input for model
Features extracted successfully
Saved features to: features/fake/musk-to-margot_segment_2012.npy
Pr

KeyboardInterrupt: 

In [16]:
# loading and preparing features for modeling 
# libraries 
import numpy as np
from pathlib import Path

# loading all real features (as .npy files)
real_features = []
for file in Path("features/real").glob("*.npy"):
    real_features.append(np.load(file))
real_features = np.array(real_features)
real_labels = np.zeros(len(real_features))  # 0 = real (target for classification)

# loading all fake features
fake_features = []
for file in Path("features/fake").glob("*.npy"):
    fake_features.append(np.load(file))
fake_features = np.array(fake_features)
fake_labels = np.ones(len(fake_features))  # 1 = fake

# combining into one array 
X = np.vstack([real_features, fake_features])
# concatenating labels into single array 
y = np.concatenate([real_labels, fake_labels])


## Train/Test Split 

* split feature data and labels into training and test sets

In [17]:
# train test sets 
from sklearn.model_selection import train_test_split

# 80/20 split 
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
) # stratify to keep original class distribution (prevent imbalance)

print(f"Train size: {len(X_train)} | Test size: {len(X_test)}")


Train size: 1249 | Test size: 313


## Defining MLP Classifier 

* Defining a Mult-layer Perceptron using PyTorch for binary classification

* Simple feed forward neural network suitable for these features

In [18]:
# defining and training MLP
import torch
import torch.nn as nn

class MLP(nn.Module):
    def __init__(self, input_dim=1024, hidden_dim=256):# size of each audio feature vector + number of neurons 
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(input_dim, hidden_dim), 
            nn.ReLU(), # ReLU activation
            nn.Dropout(0.3), # regularization
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.Dropout(0.3), 
            nn.Linear(hidden_dim, 1),
            nn.Sigmoid() # sigmoid activation to squash output between 0 and 1
        )

    def forward(self, x):
        return self.model(x)


In [ ]:
## Training MLP Classifier

In [19]:
# training 
from torch.utils.data import DataLoader, TensorDataset
import torch.optim as optim

# converting features/labels into pytorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32).unsqueeze(1)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32).unsqueeze(1)

# wraps training tensors - loaded in batches
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

# input size matching feature vector 
model = MLP(input_dim=X.shape[1])
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# adam 
optimizer = optim.Adam(model.parameters(), lr=1e-4)
criterion = nn.BCELoss() # binary cross entropy loss - binary classification

# training loop
for epoch in range(100):
    model.train()
    running_loss = 0.0
    for batch_X, batch_y in train_loader:
        batch_X, batch_y = batch_X.to(device), batch_y.to(device)

        # gradients reset
        optimizer.zero_grad()
        preds = model(batch_X)
        loss = criterion(preds, batch_y)
        loss.backward() # backpropagation
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch {epoch+1}, Loss: {running_loss:.4f}")


Epoch 1, Loss: 13.8133
Epoch 2, Loss: 13.6616
Epoch 3, Loss: 13.4606
Epoch 4, Loss: 13.2701
Epoch 5, Loss: 13.0833
Epoch 6, Loss: 12.7276
Epoch 7, Loss: 12.5999
Epoch 8, Loss: 12.2581
Epoch 9, Loss: 12.2722
Epoch 10, Loss: 12.0354
Epoch 11, Loss: 11.8338
Epoch 12, Loss: 11.7854
Epoch 13, Loss: 11.6024
Epoch 14, Loss: 11.5284
Epoch 15, Loss: 11.3750
Epoch 16, Loss: 11.2623
Epoch 17, Loss: 11.1732
Epoch 18, Loss: 11.2000
Epoch 19, Loss: 11.0583
Epoch 20, Loss: 10.9276
Epoch 21, Loss: 10.8895
Epoch 22, Loss: 10.8766
Epoch 23, Loss: 10.5583
Epoch 24, Loss: 10.5987
Epoch 25, Loss: 10.4597
Epoch 26, Loss: 10.5913
Epoch 27, Loss: 10.4248
Epoch 28, Loss: 10.2720
Epoch 29, Loss: 10.3581
Epoch 30, Loss: 10.5301
Epoch 31, Loss: 10.1393
Epoch 32, Loss: 10.0483
Epoch 33, Loss: 9.8458
Epoch 34, Loss: 10.0652
Epoch 35, Loss: 9.8185
Epoch 36, Loss: 9.9647
Epoch 37, Loss: 9.8715
Epoch 38, Loss: 9.9460
Epoch 39, Loss: 9.6478
Epoch 40, Loss: 9.5058
Epoch 41, Loss: 9.4316
Epoch 42, Loss: 9.5049
Epoch 43, 

In [ ]:
## Evaluating MLP Classifier 
* evaluating how well trained MLP performs on test set 

In [20]:
from sklearn.metrics import classification_report

# converting X_test to torch.Tensor and moved to device
X_test_tensor = torch.tensor(X_test, dtype=torch.float32).to(device)

# run inference
with torch.no_grad(): # disabling gradient tracking
    test_preds = model(X_test_tensor)
    predicted_labels = torch.round(torch.sigmoid(test_preds)).squeeze()

# print classification report
print(classification_report(y_test, predicted_labels.cpu(), target_names=["Real", "Fake"]))


              precision    recall  f1-score   support

        Real       0.00      0.00      0.00       153
        Fake       0.51      1.00      0.68       160

    accuracy                           0.51       313
   macro avg       0.26      0.50      0.34       313
weighted avg       0.26      0.51      0.35       313



/stor/home/spl742/wavenv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/stor/home/spl742/wavenv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/stor/home/spl742/wavenv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [21]:
# count class distribution in training set
from collections import Counter
print("Train distribution:", Counter(y_train))


Train distribution: Counter({np.float64(1.0): 640, np.float64(0.0): 609})
